In [78]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy.sparse.linalg
import numpy as np

In [79]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [80]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [81]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [82]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
normalized_data = StandardScaler().fit_transform(df_market_data)
#normalized_data

In [83]:
# Create a DataFrame with the scaled data
data_frame = pd.DataFrame(normalized_data, columns=['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d',	'price_change_percentage_1y'])
#data_frame.head()

# Copy the crypto names from the original data
coin_id = list(df_market_data.index) 
#coin_id

# Set the coinid column as index
data_frame['coin_id'] = coin_id  
data_frame = data_frame.set_index('coin_id') 

# Display sample data
data_frame.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [84]:
# Create a list with the number of k-values from 1 to 11
k_val = list(range(1, 12))
k_val

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [85]:
# Create an empty list to store the inertia values
inertia_val = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k_val:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(data_frame)
    inertia_val.append(k_model.inertia_)

c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

In [86]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve = {"k-values": k_val, "Inertia": inertia_val}

# Create a DataFrame with the data to plot the Elbow curve
elbow_dataframe = pd.DataFrame(elbow_curve)
elbow_dataframe.head()

,k-values,Inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,63.858668


In [10]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_dataframe.hvplot.line(
    x = "k-values",
    y = "Inertia",
    title = "Elbow Curve Plot",
    xticks= k_val
)

:Curve   [k-values]   (Inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** According to the definition the best value of k is where the data points are tightly clustered. I think is for k=4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [11]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, random_state=1)

In [12]:
# Fit the K-Means model using the scaled data
k_model.fit(data_frame)

c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=1)

In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_val = k_model.predict(data_frame)

# Print the resulting array of cluster values.
cluster_val

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0])

In [17]:
# Create a copy of the DataFrame
data_frame_copy = data_frame.copy()
data_frame_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [18]:
# Add a new column to the DataFrame with the predicted clusters
data_frame_copy['predicted_cluster'] = cluster_val

# Display sample data
data_frame_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [20]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
data_frame_copy.hvplot.scatter(
    x = 'price_change_percentage_24h',
    y = 'price_change_percentage_7d',
    by = 'predicted_cluster',
    hover_cols = 'coin_id'
)


:NdOverlay   [predicted_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [21]:
# Create a PCA model instance and set `n_components=3`.
PCA_model = PCA(n_components=3)


In [34]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
clusters_PCA = PCA_model.fit_transform(data_frame_copy)

# View the first five rows of the DataFrame. 
clusters_PCA[0:5]

array([[ 0.44890795, -1.24537573, -0.8506404 ],
       [ 0.49536709, -0.899823  , -1.31755943],
       [-0.81884571,  0.07189909,  0.69501507],
       [-0.84035711,  0.08005407,  0.54435952],
       [ 0.81324004, -2.66952035, -1.64332113]])

In [41]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.

variance = np.var(clusters_PCA) ## Pending
variance

2.3627861299834496

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Sum of variances: 2.3627861299834496

In [47]:
# Create a new DataFrame with the PCA data.
cluster_PCA_df = pd.DataFrame(
    clusters_PCA,
    columns = ["PCA_1", "PCA_2", "PCA_3"]
)
#cluster_PCA_df.head()

# Copy the crypto names from the original data
cluster_PCA_df['coin_id'] = list(data_frame_copy.index)

# Set the coinid column as index
cluster_PCA_df = cluster_PCA_df.set_index('coin_id')
#cluster_PCA_df.head()

# Display sample data
cluster_PCA_df.head()


,PCA_1,PCA_2,PCA_3
coin_id,,,
bitcoin,0.448908,-1.245376,-0.850640
ethereum,0.495367,-0.899823,-1.317559
tether,-0.818846,0.071899,0.695015
ripple,-0.840357,0.080054,0.544360
bitcoin-cash,0.813240,-2.669520,-1.643321


---

### Find the Best Value for k Using the PCA Data

In [50]:
# Create a list with the number of k-values from 1 to 11
k_val2 = list(range(1, 12))
k_val2 ## using k-val from above

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [51]:
# Create an empty list to store the inertia values
inertia_val2 = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k_val2:
    k_model2 = KMeans(n_clusters=i, random_state=1)
    k_model2.fit(cluster_PCA_df)
    inertia_val2.append(k_model2.inertia_)

c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

In [52]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve2 = {"k-values": k_val2, "Inertia": inertia_val2}

# Create a DataFrame with the data to plot the Elbow curve
elbow_dataframe2 = pd.DataFrame(elbow_curve2)
elbow_dataframe2.head()

,k-values,Inertia
0,1,290.622694
1,2,203.579726
2,3,112.854846
3,4,44.130602
4,5,33.217427


In [53]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_dataframe2.hvplot.line(
    x = "k-values",
    y = "Inertia",
    title = "Elbow Curve Plot",
    xticks= k_val2
)

:Curve   [k-values]   (Inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** According to the definition the best value of k is where the data points are tightly clustered. I think is for k=4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, in both graphs I concluded that k=4 is the best value. But it is noticed that in this graph k=4 is more tightly clustered.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [55]:
# Initialize the K-Means model using the best value for k
k_model2 = KMeans(n_clusters=4, random_state=1)


In [56]:
# Fit the K-Means model using the PCA data
k_model2.fit(cluster_PCA_df)


c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\jeny_\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=1)

In [58]:
# Predict the clusters to group the cryptocurrencies using the PCA data
cluster_val2 = k_model2.predict(cluster_PCA_df)

# Print the resulting array of cluster values.
cluster_val2

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0])

In [69]:
# Create a copy of the DataFrame with the PCA data
# Create a copy of the DataFrame
cluster_PCA_df_copy = cluster_PCA_df.copy()
#cluster_PCA_df_copy.head()

# Add a new column to the DataFrame with the predicted clusters
cluster_PCA_df_copy['predicted_cluster'] = cluster_val2

# Display sample data
cluster_PCA_df_copy.head()

,PCA_1,PCA_2,PCA_3,predicted_cluster
coin_id,,,,
bitcoin,0.448908,-1.245376,-0.850640,1
ethereum,0.495367,-0.899823,-1.317559,1
tether,-0.818846,0.071899,0.695015,0
ripple,-0.840357,0.080054,0.544360,0
bitcoin-cash,0.813240,-2.669520,-1.643321,1


In [74]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

cluster_PCA_df_copy.hvplot.scatter(
    x = 'PCA_1',
    y = 'PCA_2',
    by = 'predicted_cluster',
    hover_cols = 'coin_id'
)

# data_frame_copy.hvplot.scatter(
#     x = 'price_change_percentage_24h',
#     y = 'price_change_percentage_7d',
#     by = 'predicted_cluster',
#     hover_cols = 'coin_id'
# )


:NdOverlay   [predicted_cluster]
   :Scatter   [PCA_1]   (PCA_2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [90]:
# Composite plot to contrast the Elbow curves
elbow_dataframe.hvplot.line(
    x = "k-values",
    y = "Inertia",
    title = "Elbow Curve Plot #1",
    xticks= k_val
)


:Curve   [k-values]   (Inertia)

In [91]:
# Composite plot to contrast the Elbow curves
elbow_dataframe2.hvplot.line(
    x = "k-values",
    y = "Inertia",
    title = "Elbow Curve Plot #2",
    xticks= k_val2
)

:Curve   [k-values]   (Inertia)

In [95]:
# Composite plot to contrast the clusters
data_frame_copy.hvplot.scatter(
    x = 'price_change_percentage_24h',
    y = 'price_change_percentage_7d',
    by = 'predicted_cluster',
    hover_cols = 'coin_id'
)

:NdOverlay   [predicted_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [96]:
# Composite plot to contrast the clusters
cluster_PCA_df_copy.hvplot.scatter(
    x = 'PCA_1',
    y = 'PCA_2',
    by = 'predicted_cluster',
    hover_cols = 'coin_id'
) 

:NdOverlay   [predicted_cluster]
   :Scatter   [PCA_1]   (PCA_2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Although both elbow plots, k-value is the same, in the PCA elbow plot it can be visualized that custers are more tight.